In [47]:
from mcomp import M1, M3, load_m1, load_m3

from smooth import ADAM, ES

import numpy as np
import pandas as pd


In [ ]:
series = M3[2568]


model = ES(model="MAM", lags=[series['period']])
model_fit = model.fit(series['x'])
print(model_fit)
# forecasts = model.predict(h=18)
# forecasts

12
Time elapsed: 0.08 seconds
Model estimated using ES() function: ETS(MAM)
With backcasting initialisation
Distribution assumed in the model: Normal
Loss function type: likelihood; Loss function value: 869.8277
Persistence vector g:
 alpha   beta  gamma
0.0209 0.0208 0.0852
Sample size: 116
Number of estimated parameters: 3
Number of degrees of freedom: 113
Information criteria:
      AIC      AICc       BIC      BICc
1745.6554 1745.8697 1753.9161 1754.4254
